In [1]:
import pandas as pd
import os

In [7]:
root_dir = os.path.dirname(os.getcwd())
print(root_dir)
orig_sample_data_dir = os.path.join(root_dir, 'Sample Data')
print(orig_sample_data_dir)
tsv_files = [f for f in os.listdir(orig_sample_data_dir) if f.split('.', 1)[1] == 'tsv']
print(tsv_files)

tsv_file_length = []
for f in tsv_files:
    df = f.split('.', 1)[0]
    df = pd.read_csv(os.path.join(orig_sample_data_dir, f), sep='\t')
    tsv_file_length.append(df.shape[0])
    print(df.head())

print(tsv_files)
print(tsv_file_length)

    

/Users/mengruf/Documents/Github/OMSCS/CS6400_FALL20/project/cs6400-2020-03-Team040/Phase_3/service/db
/Users/mengruf/Documents/Github/OMSCS/CS6400_FALL20/project/cs6400-2020-03-Team040/Phase_3/service/db/Sample Data
['store.tsv', 'product.tsv', 'product_sale_prices.tsv', 'category.tsv', 'city.tsv', 'manufacturer.tsv', 'membership.tsv', 'holiday.tsv', 'product_sales.tsv', 'date.tsv']
   storeid           phone                        address     cityname state
0        1  (239) 167-4627              78 Fabien Freeway      Oakland    NH
1        2  (744) 032-1717             29 West Milton St.  New Orleans    IN
2        3    544-510-8934             271 Fabien Parkway      Lubbock    IL
3        4    340-264-5348  444 North Rocky Milton Avenue    Cleveland    CT
4        5  (111) 026-2512                 97 Hague Blvd.        Tampa    NE
   productid           name                 Manufacturer  retailprice  \
0          1       Zeejubin    Bartanefin International        258.36   
1     

In [8]:
# Date.csv 
def transform_date_data(input_file): 
    curr_dir = os.getcwd()
    root_dir = os.path.dirname(os.getcwd())
    orig_sample_data_dir = os.path.join(root_dir, 'Sample Data')
    # read tsv file
    df = pd.read_csv(os.path.join(orig_sample_data_dir, input_file), sep='\t')
    # use index as dateID
    df['dateID'] = df.index
    # rename column name
    df = df.rename({'date': 'date_time', 'dateID': 'dateID'}, axis=1)
    # sort columns
    df = df[['dateID', 'date_time']]

    print(df.head())
    print(df.shape[0])

    # write into sample_data folder
    df.to_csv(os.path.join(curr_dir, 'Date.csv'), index=False)
    
transform_date_data('date.tsv')   

   dateID   date_time
0       0  2000-01-01
1       1  2000-01-02
2       2  2000-01-03
3       3  2000-01-04
4       4  2000-01-05
4566


In [9]:
# Create Date Map
def output_date_data(input_file): 
    curr_dir = os.getcwd()
    root_dir = os.path.dirname(os.getcwd())
    orig_sample_data_dir = os.path.join(root_dir, 'Sample Data')
    # read tsv file
    df = pd.read_csv(os.path.join(orig_sample_data_dir, input_file), sep='\t')
    # use index as dateID
    df['dateID'] = df.index
    # rename columns
    df = df.rename({'date': 'date_time', 'dateID': 'dateID'}, axis=1)
    # sort columns
    df = df[['dateID', 'date_time']]
    
    return df 

output_date_data('date.tsv')

,dateID,date_time
0,0,2000-01-01
1,1,2000-01-02
2,2,2000-01-03
3,3,2000-01-04
4,4,2000-01-05
...,...,...
4561,4561,2012-06-27
4562,4562,2012-06-28
4563,4563,2012-06-29
4564,4564,2012-06-30


In [28]:
# Holiday.csv, NonHoliday.csv
def transform_holiday_data(input_file, date_map): 
     curr_dir = os.getcwd()
     root_dir = os.path.dirname(os.getcwd())
     orig_sample_data_dir = os.path.join(root_dir, 'Sample Data')
     # read tsv file
     df = pd.read_csv(os.path.join(orig_sample_data_dir, input_file), sep='\t')
     # rename columns
     df = df.rename({'date': 'date_time', 'holidayname': 'holiday_name'}, axis=1)
     # map dateID column from date data
     df_merged = pd.merge(df, date_map, on = ['date_time'], how = 'left')
     # remove date_time column
     df_holiday = df_merged[['dateID', 'holiday_name']]

     print(df_holiday.head())
     print(df_holiday.shape[0])

     # write into sample_data folder
     df_holiday.to_csv(os.path.join(curr_dir, 'Holiday.csv'), index=False)

     # nonholiday 
     df_nonholiday = pd.merge(date_map, df, on = ['date_time'], how = 'left')
     df_nonholiday = df_nonholiday.loc[df_nonholiday['holiday_name'].isnull(), :]
     df_nonholiday = df_nonholiday[['dateID']]

     print(df_nonholiday.head())
     print(df_nonholiday.shape[0])
     
     # write into sample_data folder
     df_nonholiday.to_csv(os.path.join(curr_dir, 'NonHoliday.csv'), index=False) 
     

     

transform_holiday_data('holiday.tsv', output_date_data('date.tsv'))
    

   dateID       holiday_name
0    4018     New Year's Day
1    4034  Martin Luther Day
2    4069    President's Day
3    4167       Memorial Day
4    4202   Independence Day
15
   dateID
0       0
1       1
2       2
3       3
4       4
4551


In [12]:
# Sold.csv 
def transform_sold_data(input_file, date_map): 
    curr_dir = os.getcwd()
    root_dir = os.path.dirname(os.getcwd())
    orig_sample_data_dir = os.path.join(root_dir, 'Sample Data')
    # read tsv file
    df = pd.read_csv(os.path.join(orig_sample_data_dir, input_file), sep='\t')
    # rename columns 
    df = df.rename({'productid': 'PID', 'storeid': 'store_number', 'date': 'date_time', 'quantity': 'quantity'}, axis=1)
    # map dateID column from date data
    df_merged = pd.merge(df, date_map, on = ['date_time'], how = 'left')
    # remove date_time column
    df_truncated = df_merged[['store_number', 'dateID', 'PID', 'quantity']]
    
    print(df_truncated.head())
    print(df_truncated.shape[0])

    # write into sample_data folder
    df_truncated.to_csv(os.path.join(curr_dir, 'Sold.csv'), index=False)

transform_sold_data('product_sales.tsv', output_date_data('date.tsv'))

   store_number  dateID  PID  quantity
0           108    2577    1         1
1           214     975    1         7
2           234     861    1         2
3           242     257    1         1
4           315    1884    1         3
500000


In [14]:
# Membership.csv, YellowJacket.csv, GiantHornet.csv
def transform_membership_data(input_file, date_map): 
    curr_dir = os.getcwd()
    root_dir = os.path.dirname(os.getcwd())
    orig_sample_data_dir = os.path.join(root_dir, 'Sample Data')
    # read tsv file
    df = pd.read_csv(os.path.join(orig_sample_data_dir, input_file), sep='\t')
    # rename columns 
    df = df.rename({'membershipid': 'memberID', 'membershiptype': 'membershiptype', 'storeid': 'store_number', 'date': 'date_time'}, axis=1)
    # map dateID column from date data
    df_merged = pd.merge(df, date_map, on = ['date_time'], how = 'left')
    # remove date_time column
    df_membership = df_merged[['memberID', 'dateID', 'store_number']]
    # rename dateID to signup_date 
    df_membership = df_membership.rename({'dateID': 'signup_date'}, axis=1)

    # subclass: YellowJacket, GiantHornet
    df_yellowjacket = df_merged.loc[df_merged['membershiptype'] == 'Yellow Jacket', ['memberID']]
    df_gianthornet = df_merged.loc[df_merged['membershiptype'] == 'Giant Hornet', ['memberID']]

    print(df_membership.head())
    print(df_membership.shape[0])
    print(df_yellowjacket.head())
    print(df_yellowjacket.shape[0])
    print(df_gianthornet.head())
    print(df_gianthornet.shape[0])

    # write into sample_data folder
    df_membership.to_csv(os.path.join(curr_dir, 'Membership.csv'), index=False)
    df_yellowjacket.to_csv(os.path.join(curr_dir, 'YellowJacket.csv'), index=False)
    df_gianthornet.to_csv(os.path.join(curr_dir, 'GiantHornet.csv'), index=False)

transform_membership_data('membership.tsv', output_date_data('date.tsv'))
    

   memberID  signup_date  store_number
0    100000          976           437
1    100001         4035           150
2    100002          359           114
3    100003         4472           265
4    100004         3179           506
400000
   memberID
0    100000
1    100001
3    100003
4    100004
5    100005
300041
    memberID
2     100002
6     100006
7     100007
11    100011
15    100015
99959


In [18]:
# validate how many YellowJacket and GiantHornet in original tsv file 
membership_df = pd.read_csv(os.path.join(orig_sample_data_dir, 'membership.tsv'), sep='\t')
print(membership_df.head())

membership_df.groupby('membershiptype')['membershipid'].count()

   membershipid membershiptype  storeid        date
0        100000  Yellow Jacket      437  2002-09-03
1        100001  Yellow Jacket      150  2011-01-18
2        100002   Giant Hornet      114  2000-12-25
3        100003  Yellow Jacket      265  2012-03-30
4        100004  Yellow Jacket      506  2008-09-14


membershiptype
Giant Hornet      99959
Yellow Jacket    300041
Name: membershipid, dtype: int64

In [37]:
# OnSale.csv
def transform_onsale_data(input_file1, input_file2, input_file3, date_map): 
    curr_dir = os.getcwd()
    root_dir = os.path.dirname(os.getcwd())
    orig_sample_data_dir = os.path.join(root_dir, 'Sample Data')
    # read tsv file
    df1 = pd.read_csv(os.path.join(orig_sample_data_dir, input_file1), sep='\t')
    df2 = pd.read_csv(os.path.join(orig_sample_data_dir, input_file2), sep='\t')
    df3 = pd.read_csv(os.path.join(orig_sample_data_dir, input_file3), sep='\t')

    # only keep sold product
    df_sold = pd.merge(df1, df3, on = ['productid'], how = 'inner')

    # truncate unused columns
    df_sold = df_sold[['productid', 'date', 'retailprice']]
    # drop duplicated rows 
    df_sold = df_sold.drop_duplicates()

    print(df_sold.shape)
    print(df_sold.head())

    df_merged = pd.merge(df_sold, df2, on = ['productid', 'date'], how = 'left')

    df_merged['discount_percentage'] = df_merged['saleprice'].fillna(0) / df_merged['retailprice'].fillna(0)
    
    df_truncated = df_merged[['productid', 'date', 'discount_percentage']]

    # rename columns 
    df_truncated = df_truncated.rename({'productid': 'PID', 'date':'date_time'}, axis=1)
 
    # map dateID column from date data
    df_mapped = pd.merge(df_truncated, date_map, on = ['date_time'], how = 'left')
    # remove date_time column
    df_clean = df_mapped[['PID', 'dateID', 'discount_percentage']]

    print(df_clean.head())
    print(df_clean.shape[0])
    
    # discount_percentage = 0 also shows in OnSale table
    # print(df_clean.loc[df_clean['discount_percentage'] == 0, :].head())
    print(df_clean.loc[df_clean['discount_percentage'] == 0, :].shape[0])

    print(df_clean.loc[df_clean['discount_percentage'] > 0, :].shape[0]) # matched with product_sale_prices.tsv

    # write into sample_data folder
    df_clean.to_csv(os.path.join(curr_dir, 'OnSale.csv'), index=False)

transform_onsale_data('product_sales.tsv', 'product_sale_prices.tsv', 'product.tsv', output_date_data('date.tsv'))

(499023, 3)
   productid        date  retailprice
0          1  2007-01-21       258.36
1          1  2002-09-02       258.36
2          1  2002-05-11       258.36
3          1  2000-09-14       258.36
4          1  2005-02-27       258.36
   PID  dateID  discount_percentage
0    1    2577                  0.0
1    1     975                  0.0
2    1     861                  0.0
3    1     257                  0.0
4    1    1884                  0.0
499023
469515
29508


In [42]:
# validate product_sales.tsv 
product_sales_df = pd.read_csv(os.path.join(orig_sample_data_dir, 'product_sales.tsv'), sep='\t')
print(product_sales_df.head())
print(product_sales_df.shape)

product_sales_df1 = product_sales_df[['productid', 'date']].drop_duplicates()
print(product_sales_df1.shape)

product_sales_df2 = product_sales_df[['productid', 'date', 'quantity']].drop_duplicates()
print(product_sales_df2.shape)


#membership_df.groupby('membershiptype')['membershipid'].count()

   productid  storeid        date  quantity
0          1      108  2007-01-21         1
1          1      214  2002-09-02         7
2          1      234  2002-05-11         2
3          1      242  2000-09-14         1
4          1      315  2005-02-27         3
(500000, 4)
(499023, 2)
(499894, 3)
